## Fase 3.1 — Ingestão de Dados Reais (KaggleHub)

Nesta etapa damos sequência à **Fase 3.1** do plano de atividades, que corresponde à primeira ação prática com dados reais, agora utilizando o **KaggleHub** para baixar o dataset diretamente do repositório oficial do Kaggle.  
Essa abordagem garante que o arquivo seja obtido na versão mais recente, com rastreabilidade e compatibilidade para versionamento com DVC. O processo será executado **dentro do container**, evitando divergências entre ambiente local e remoto.

Após o download, o caminho de saída será exibido para validação, permitindo o rastreamento do arquivo bruto e preparação para push no backend MinIO/DVC.


In [13]:
# ETAPA: DOWNLOAD DOS DADOS REAIS VIA KAGGLEHUB

# Instala o kagglehub dentro do ambiente do notebook (caso ainda não esteja instalado)
%pip install kagglehub

# Importa a biblioteca
import kagglehub

print("Pacote kagglehub importado com sucesso.")

# Baixa a versão mais recente do dataset do Kaggle
path = kagglehub.dataset_download("parisrohan/credit-score-classification")

print("\nDownload concluído.")
print("Caminho local do dataset:", path)

# Lista o conteúdo real do diretório baixado para validar o nome exato
import os

print("\nConteúdo real do path baixado:")
print(os.listdir(path))



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Pacote kagglehub importado com sucesso.

Download concluído.
Caminho local do dataset: /root/.cache/kagglehub/datasets/parisrohan/credit-score-classification/versions/1

Conteúdo real do path baixado:
['test.csv', 'train.csv']


In [15]:
# ETAPA: MOVER ARQUIVOS USANDO O PATH REAL, FORÇANDO DESTINO NO RAIZ DO REPOSITÓRIO

import os
import shutil
from pathlib import Path

# Caminho de origem
source_dir = path  # valor real do kagglehub

# Força caminho para /workspace/data/raw
repo_root = Path.cwd().parent  # se notebook está em /notebooks/, sobe um nível
destination_dir = repo_root / "data" / "raw"
destination_dir.mkdir(parents=True, exist_ok=True)

print("Destino absoluto:", destination_dir)

# Lista source_dir antes
print("Conteúdo em source_dir:", os.listdir(source_dir))

# Copia tudo
for filename in os.listdir(source_dir):
    source_file = os.path.join(source_dir, filename)
    destination_file = destination_dir / filename
    shutil.copy2(source_file, destination_file)
    print(f"Copiado: {filename}")

print("Conteúdo atual em data/raw/:", os.listdir(destination_dir))


Destino absoluto: /workspace/data/raw
Conteúdo em source_dir: ['test.csv', 'train.csv']
Copiado: test.csv
Copiado: train.csv
Conteúdo atual em data/raw/: ['test.csv', 'train.csv']


In [21]:
# ETAPA: VERSIONAR ARQUIVOS COM DVC ADD (CORRIGIDO PARA CWD /workspace/notebooks)

# Verifica onde está o kernel e o conteúdo real
import os
print("CWD Kernel:", os.getcwd())
!pwd
!ls -lh ../data/raw

# Executa o dvc add no caminho real (um nível acima)
!dvc add ../data/raw

# Lista o conteúdo para conferir arquivos .dvc criados
!ls -lh ../data/raw
!cat ../data/raw.dvc


CWD Kernel: /workspace/notebooks
/workspace/notebooks
total 45M
-rw-r--r-- 1 root root 15M Jul 14 09:17 test.csv
-rw-r--r-- 1 root root 30M Jul 14 09:17 train.csv
 ⠋ Checking graph
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/workspace/.dvc/cache/files/md5'| |0/? [00:00<?,    ?fil
                                                                                
!
Checking out /workspace/data/raw                      |0.00 [00:00,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 52.46file/s]
total 45M
-rw-r--r-- 1 root root 15M Jul 14 09:17 test.csv
-rw-r--r-- 1 root root 30M Jul 14 09:17 train.csv
outs:
- md5: 5dc4156c8cba026c76905a26de89fdb9.dir
  size: 46502530
  nfiles: 2
  hash: md5
  path: raw


In [22]:
# ETAPA: ENVIAR ARQUIVOS VERSIONADOS PARA O BACKEND REMOTO COM DVC PUSH

# Executa push dos dados versionados para o remote configurado no DVC
!dvc push

# Verifica status para garantir que tudo foi enviado corretamente
!dvc status


Pushing
!
  0% Querying remote cache|                          |0/1 [00:00<?,    ?files/s]
100% Querying remote cache|█████████████████████|1/1 [00:00<00:00,  4.11files/s]
                                                                                
!
  0% Querying remote cache|                          |0/0 [00:00<?,    ?files/s]
Pushing                                                                         
Everything is up to date.
Data and pipelines are up to date.                                              
